In [ ]:
#  ╔═══╦═════════╗
#  ║ 1 ║ SET UP  ║
#  ╚═══╩═════════╝


# Dash packages
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State, ALL
import plotly.graph_objects as go


# Leaflet packages
import dash_leaflet as dl
from dash_leaflet import express as dlx
import kpi_heatmaps as kh


# Data packages
import pandas as pd
import json


# Baixar JSON
with open('assets/SC_geo.json', 'r') as sc:
    geo = json.load(sc)
    

# Paleta de cores
marks = [0, 10, 20, 30, 60, 70, 80, 90]
colorscale = ['#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026']


# Indicadores
style_functions = {
    'Lixo Coletado': kh.get_lixo,
    'Energia Elétrica': kh.get_energia,
    'Rede Geral de Abastecimento de Água': kh.get_agua,
    'Sem Banheiro ou Sanitário': kh.get_sanitario
}

KPIS = list(style_functions.keys())


def get_info(feature=None, kpi=None):
    header = [html.H4("Santa Catarina")]
    if not feature:
        return header + ["Escolha um município"]
    return header + [html.B(feature["properties"]["name"]), html.Br(),
                     f'{kpi}: {float(feature["properties"][kpi])}%']




#  ╔═══╦════════════╗
#  ║ 2 ║ FRONT END  ║
#  ╚═══╩════════════╝


# Criar Dash App
app = dash.Dash(external_stylesheets=[dbc.themes.SANDSTONE])
app.title = 'Indicadores Municipais'
app.layout = html.Div([
    
    dcc.Store('memory_kpi', data='Lixo Coletado'),
    
    dbc.NavbarSimple([        
        dbc.DropdownMenu([dbc.DropdownMenuItem(kpi, id={'item_kpi': kpi}) for kpi in KPIS],
                         direction='left',
                         nav=True,
                         in_navbar=True,
                         label='Indicador',
                         id='button_kpi')
    ],
    brand="Indicadores Municipais",
    brand_href="#",
    color="primary",
    dark=True,
),
    
    dbc.Container([
        dbc.Row(style={'width': '100%', 'height': '90vh', 'margin': "auto", "display": "block"}, id="map")
    ])
    
])




#  ╔═══╦═════════╗
#  ║ 3 ║ SERVER  ║
#  ╚═══╩═════════╝


# Carregar mapa
@app.callback([Output('map', 'children'),
               Output('button_kpi', 'label'),
               Output('memory_kpi', 'data')],
             [Input({'item_kpi': ALL}, 'n_clicks')])
def set_kpi(click):
    
    # Identificar botão apertado
    kpi = dash.callback_context.triggered[0]['prop_id'][13:-11]
    if kpi == '':
        kpi = KPIS[0]
    
    # Criar colorbar
    ctg = ["{}+".format(mark, marks[i + 1]) for i, mark in enumerate(marks[:-1])] + ["{}+".format(marks[-1])]
    colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")
    
    # Criar geojson
    options = {'hoverStyle': {'weight': 5, 'color': '#666', 'dashArray': ''}, 'zoomToBoundsOnClick': True}
    geojson = dlx.geojson(geo, id="geojson", defaultOptions=options, style=style_functions[kpi])
    #geojson = dl.GeoJSON(geo, id="geojson", defaultOptions=options, featureStyle=feature_style, featureId=feature_id)
    
    # Criar caixa de informações
    info = html.Div(children=get_info(), id="info", className="info",
                    style={"position": "absolute", "bottom": "10px", "right": "10px", "z-index": "1000"})
    
    return dl.Map(children=[dl.TileLayer(), geojson, colorbar, info], center=[-27, -51], zoom=7.3), kpi, kpi


# Atualizar caixa de informações
@app.callback(Output("info", "children"),
             [Input("geojson", "featureHover")],
             [State('memory_kpi', 'data')])
def info_hover(feature, data):
    return get_info(feature, kpi=data)


# Rodar server
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jul/2020 16:27:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_1m1593643348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET /assets/style.css?m=1593649126.0682344 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_1m1593643348.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_1m1593643348.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_1m1593643348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 16:27:39] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1593643370.min.js HTTP/1.1" 200 -
12